<a href="https://colab.research.google.com/github/r-matsuzaka/practice-elements-of-programming/blob/main/colab/chapter_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter16 Parallel Computing

In [1]:
%%capture
!pip install yapf

from IPython.core.magic import register_cell_magic
from yapf.yapflib.yapf_api import FormatCode


@register_cell_magic
def fmt(line, cell):
   """
   My formatter cell magic comannd.
   Please install yapf before using this magic command.
   """
   print(FormatCode(cell, style_config='pep8')[0])

In [2]:
# %%fmt
from time import sleep, perf_counter
from threading import Thread


def task():
    print('Starting a task...')
    sleep(1)
    print('done')


start_time = perf_counter()

# create two new threads
t1 = Thread(target=task)
t2 = Thread(target=task)

# start the threads
t1.start()
t2.start()

# wait for the threads to complete
t1.join()
t2.join()

end_time = perf_counter()

print(f'It took {end_time- start_time: 0.2f} second(s) to complete.')

Starting a task...
Starting a task...
done
done
It took  1.00 second(s) to complete.


In [3]:
from time import sleep, perf_counter
from threading import Thread


def task(id):
    print(f'Starting the task {id}...')
    sleep(1)
    print(f'task {id} done')


start_time = perf_counter()

# create and start 10 threads
threads = []
for n in range(1, 10):
    t = Thread(target=task, args=(n,))
    threads.append(t)
    t.start()

# wait for the threads to complete
for t in threads:
    t.join()

end_time = perf_counter()

print(f'It took {end_time- start_time: 0.2f} second(s) to complete.')

Starting the task 1...
Starting the task 2...
Starting the task 3...
Starting the task 4...Starting the task 5...

Starting the task 6...
Starting the task 7...
Starting the task 8...
Starting the task 9...
task 1 done
task 2 done
task 3 done
task 5 done
task 4 done
task 6 donetask 7 done

task 8 done
task 9 done
It took  1.01 second(s) to complete.


In [4]:
#%%fmt
from concurrent.futures import ThreadPoolExecutor
import time
import threading

from abc import ABC, abstractmethod


class AbstractLock(ABC):

    def __init__(self, lock):
        self.lock = lock

    @abstractmethod
    def run(self):
        pass


class Lock1(AbstractLock):

    def __init__(self, lock):
        super().__init__(lock)

    def run(self):
        while True:
            self.lock.acquire()  # 排他制御開始
            print("Lock1 : lock acquire")
            time.sleep(1)
            print("Lock1 : lock release")
            self.lock.release()  # 排他制御解除
            time.sleep(1)


class Lock2(AbstractLock):

    def __init__(self, lock):
        super().__init__(lock)

    def run(self):
        while True:
            self.lock.acquire()  # 排他制御開始
            print("Lock2 : lock acquire")
            time.sleep(1)
            print("Lock2 : lock release")
            self.lock.release()  # 排他制御解除
            time.sleep(1)


class LockFactory():

    def create_lock(self, name, lock):
        targetclass = name.capitalize()

        return globals()[targetclass](lock)


def main():

    th_pool = []
    lock = threading.Lock()  # Lockモジュール生成

    lock_factory = LockFactory()
    lock_names = ['lock1', 'lock2']

    for name in lock_names:
        lock_obj = lock_factory.create_lock(name, lock)
        th_pool.append(lock_obj.run)  # 各スレッドにLockモジュールを引数として渡す

    with ThreadPoolExecutor(max_workers=2) as executor:
        for th in th_pool:
            executor.submit(th)  # スレッド生成


main()

Lock1 : lock acquire
Lock1 : lock release
Lock2 : lock acquire
Lock2 : lock release
Lock1 : lock acquire
Lock1 : lock release
Lock2 : lock acquire
Lock2 : lock release
Lock1 : lock acquire
Lock1 : lock release
Lock2 : lock acquire


KeyboardInterrupt: ignored

Lock2 : lock release
Lock1 : lock acquire


# Reference
- [Python Threading](https://www.pythontutorial.net/advanced-python/python-threading/)  
- [What’s are the Differences between Processes and Threads](https://www.pythontutorial.net/advanced-python/differences-between-processes-and-threads/)
- [python3: Threadで排他制御をする](https://www.raspberrypirulo.net/entry/thread-lock)